# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [8]:
#import warnings
#warnings.filterwarnings('ignore')

useLogging = False # set to True to get logging information (and hopefully track which LLM is called when)
use_Ollama_For_Inference = True # set to True to use Ollama inference models (and pull at least the gemma:2b model)
use_Ollama_For_Embedding = True # set to True to use Ollama embedddings models (and pull at least the nomic-embed-text:latest model)
use_Pinecone = False #Turn on to use a Pinecone database. Sign up at www.pinecone.io for a free plan (including 5 indexes)
use_Chroma = True #Turn on to use a local Chroma database. Supersedes the use_Pinecone flag above (and turns it off)
use_Test_Data = False # set to True to use LimitedCSVLoader class below and only load the 577th item from the CSV file and test that querying with embeddings work well.

import openai
#Defaults to OpenAI if use_Ollama_For_Inference=False and use_Ollama_For_Inference=False
openai.api_base = inferApiBase = embedApiBase =  "https://api.openai.com/v1"
openai.base_url = inferBaseUrl = embedBaseUrl = "https://api.openai.com"
openai.api_key = inferApiKey = embedApiKey = os.environ['OPENAI_API_KEY']
embeddings_model_name = "text-embedding-ada-002"
embeddings_model_name_short = "ada"
embeddings_vector_size = 1536
infer_model_name = "gpt-3.5-turbo"
llm_platform = "openai"
embed_chunk_size = 1000
embed_overlap = 0


if use_Ollama_For_Inference:
    inferApiBase = "http://localhost:11434/v1"
    inferBaseUrl = "http://localhost:1143"
    inferApiKey = "ollama"
    infer_model_name = "gemma:2b" #you can/should customize this to test different Ollama LLMs. Use the NAME field from `ollama list`


if use_Ollama_For_Embedding:
    llm_platform = "ollama"
    embedApiBase = "http://localhost:11434/v1"
    embedBaseUrl = "http://localhost:1143"
    embedApiKey = "ollama"
    #embeddings_model_name = "mxbai-embed-large:latest"
    #embeddings_model_name_short = "mxbai"
    #embeddings_vector_size = 1024
    #embed_chunk_size = 512
    #embed_overlap = 10
    
    embeddings_model_name = "nomic-embed-text:latest" #you can/should customize this to test different Ollama embedding LLMs. Use the NAME field from `ollama list`
    embeddings_model_name_short = "nomic"
    embeddings_vector_size = 768
    embed_chunk_size = 8192
    embed_overlap = 0

print('Embed API Key:', embedApiKey)
print('Infer API Key:', inferApiKey)
print('Embed API Base:', embedApiBase)
print('Infer API Base:', inferApiBase)
print('Embeddings Model:', embeddings_model_name)
print('Inference Model:', infer_model_name)

from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings


if use_Ollama_For_Embedding:
    embeddings_model = OllamaEmbeddings(model=embeddings_model_name, embed_instruction='', query_instruction='')
    #embeddings_model = OllamaEmbeddings(model=embeddings_model_name)
else:
    embeddings_model = OpenAIEmbeddings(model=embeddings_model_name)

if useLogging:
    import logging
    import requests 


    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s - %(levelname)s - %(filename)s:%(funcName)s:%(lineno)d - %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')
    requests.packages.urllib3.add_stderr_logger()
    OLLAMA_DEBUG=1
    langchain.debug = True

Embed API Key: ollama
Infer API Key: ollama
Embed API Base: http://localhost:11434/v1
Infer API Base: http://localhost:11434/v1
Embeddings Model: nomic-embed-text:latest
Inference Model: gemma:2b


In [9]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
#!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [4]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
llm = ChatOpenAI(temperature=0.9, base_url=inferApiBase, model=infer_model_name)


In [12]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [8]:

chain = prompt | llm

In [ ]:
product = "Queen Size Sheet Set"
chain.invoke(product)

## SimpleSequentialChain

In [10]:
from langchain.chains import SimpleSequentialChain

In [4]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

h:\Users\Raphael\OneDrive\Perso\Technical\AI\Training\GenAI\LangChain-for-LLM-Application-Development\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [17]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [20]:
#Code to run with the newer RunnableSequence implementation for Chains

from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

chain_one = first_prompt | llm | StrOutputParser()
chain_two = second_prompt | llm | StrOutputParser()
overall_simple_chain = ({"company_name" : chain_one} 
         | RunnablePassthrough.assign(review=chain_two)
        )

In [ ]:
overall_simple_chain.invoke(product)

## SequentialChain

In [10]:
from langchain.chains import SequentialChain
from langchain.schema import StrOutputParser

2024-06-10 22:19:33,239 DEBUG Added a stderr logging handler to logger: urllib3
2024-06-10 22:19:33,239 DEBUG Added a stderr logging handler to logger: urllib3
2024-06-10 22:19:33 - DEBUG - __init__.py:add_stderr_logger:90 - Added a stderr logging handler to logger: urllib3


Embed API Key: ollama
Infer API Key: ollama
Embed API Base: http://localhost:11434/v1
Infer API Base: http://localhost:11434/v1
Embeddings Model: mxbai-embed-large:latest
Inference Model: gemma:2b


In [ ]:
llm = ChatOpenAI(temperature=0.9)

In [128]:
from langchain_openai import ChatOpenAI
infer_model_name = "llama3:8b"
llm = ChatOpenAI(temperature = 0.0, base_url=inferApiBase, model=infer_model_name)

2024-06-10 23:40:26 - DEBUG - _config.py:load_ssl_context:80 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-06-10 23:40:26 - DEBUG - _config.py:load_ssl_context_verify:146 - load_verify_locations cafile='h:\\Users\\Raphael\\OneDrive\\Perso\\Technical\\AI\\Training\\GenAI\\LangChain-for-LLM-Application-Development\\venv\\lib\\site-packages\\certifi\\cacert.pem'
2024-06-10 23:40:26 - DEBUG - _config.py:load_ssl_context:80 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-06-10 23:40:26 - DEBUG - _config.py:load_ssl_context_verify:146 - load_verify_locations cafile='h:\\Users\\Raphael\\OneDrive\\Perso\\Technical\\AI\\Training\\GenAI\\LangChain-for-LLM-Application-Development\\venv\\lib\\site-packages\\certifi\\cacert.pem'


In [212]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:"
    "\n\n{Review}. Do not mention in the response that this is a translation."
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [213]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [214]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}. Respond with one single word, such as Portuguese, German or French."
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [219]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "As a customer service representative, write a follow up response to the following "
    "review summary in the specified language:"
    "\n\nReview Summary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [220]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [222]:
langchain.debug = False
review = df.Review[5]
overall_chain.invoke(review)

2024-06-11 01:10:11 - DEBUG - _base_client.py:_build_request:446 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': "Translate the following review to English:\n\nJe trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?. Do not mention in the response that this is a translation.", 'role': 'user'}], 'model': 'llama3:8b', 'n': 1, 'stream': False, 'temperature': 0.0}}
2024-06-11 01:10:11 - DEBUG - _base_client.py:_request:949 - Sending HTTP Request: POST http://localhost:11434/v1/chat/completions
2024-06-11 01:10:11 - DEBUG - _trace.py:trace:45 - close.started
2024-06-11 01:10:11 - DEBUG - _trace.py:trace:45 - close.complete
2024-06-11 01:10:11 - DEBUG - _trace.py:trace:45 - connect_tcp.started host='localhost' port=11434 local_address=None timeout=None socket_options=None




> Entering new SequentialChain chain...


2024-06-11 01:10:13 - DEBUG - _trace.py:trace:45 - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x00000293F15E9EA0>
2024-06-11 01:10:13 - DEBUG - _trace.py:trace:45 - send_request_headers.started request=<Request [b'POST']>
2024-06-11 01:10:13 - DEBUG - _trace.py:trace:45 - send_request_headers.complete
2024-06-11 01:10:13 - DEBUG - _trace.py:trace:45 - send_request_body.started request=<Request [b'POST']>
2024-06-11 01:10:13 - DEBUG - _trace.py:trace:45 - send_request_body.complete
2024-06-11 01:10:13 - DEBUG - _trace.py:trace:45 - receive_response_headers.started request=<Request [b'POST']>
2024-06-11 01:10:19 - DEBUG - _trace.py:trace:45 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json'), (b'Date', b'Mon, 10 Jun 2024 23:10:19 GMT'), (b'Content-Length', b'444')])
2024-06-11 01:10:19 - INFO - _client.py:_send_single_request:1026 - HTTP Request: POST http://localhost:11434/v1/chat/complet


> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The mousse doesn't hold up, it's weird. I buy the same ones in stores and the taste is much better...\n\nOld batch or counterfeit!?!",
 'summary': "The reviewer finds the taste of their purchased product to be mediocre and inferior to what they've experienced with the same product from stores, leading them to speculate that it might be an old batch or even a counterfeit.",
 'followup_message': "Bonjour,\n\nJe suis désolé d'apprendre que le goût de notre produit ne vous a pas satisfait. Nous comprenons que la qualité du goût est essentielle pour nous et nos clients, et nous sommes prêts à prendre des mesures pour résoudre ce problème.\n\nPouvez-vous me donner plus de détails sur votre expérience avec notre produit ? Quel est le nom du produit en question et pouvez

In [100]:
#import langchain
#langchain.debug = True
review = df.Review[5]
overall_chain.invoke(review)

2024-06-10 22:40:40 - DEBUG - _base_client.py:_build_request:446 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': "Translate the following review to english:\n\nJe trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?", 'role': 'user'}], 'model': 'gpt-3.5-turbo', 'n': 1, 'stream': False, 'temperature': 0.9}}
2024-06-10 22:40:40 - DEBUG - _base_client.py:_request:949 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
2024-06-10 22:40:40 - DEBUG - _trace.py:trace:45 - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2024-06-10 22:40:40 - DEBUG - _trace.py:trace:45 - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x00000293EF4BE6E0>
2024-06-10 22:40:40 - DEBUG - _trace.py:trace:45 - start_tls.started ssl_c

[chain/start] [chain:SequentialChain] Entering Chain run with input:
{
  "Review": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?"
}
[chain/start] [chain:SequentialChain > chain:LLMChain] Entering Chain run with input:
{
  "Review": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?"
}
[llm/start] [chain:SequentialChain > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Translate the following review to english:\n\nJe trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?"
  ]
}


2024-06-10 22:40:41 - DEBUG - _trace.py:trace:45 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 10 Jun 2024 20:40:41 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-vycexojfpphtlaesrwlkvocv'), (b'openai-processing-ms', b'738'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'60000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'59929'), (b'x-ratelimit-reset-requests', b'8.64s'), (b'x-ratelimit-reset-tokens', b'71ms'), (b'x-request-id', b'req_729fbbe71d06b24b7760256dbb17f7b2'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=2FY63cnHrQJw23x6rp88EDb209kfbgf1nk.VNzAmTB4-1718052041-1.0.1.1-sVWm._5OLcFDmzsFA.vWRe2me5SGPBEETkZLryf9I875hHv2UDXtAi9mU6cjUl6rvVOFhPSadmsc5W

[llm/end] [chain:SequentialChain > chain:LLMChain > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I find the taste mediocre. The foam does not hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "I find the taste mediocre. The foam does not hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 35,
                "prompt_tokens": 64,
               

2024-06-10 22:40:41 - DEBUG - _trace.py:trace:45 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 10 Jun 2024 20:40:42 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-vycexojfpphtlaesrwlkvocv'), (b'openai-processing-ms', b'388'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'60000'), (b'x-ratelimit-remaining-requests', b'9998'), (b'x-ratelimit-remaining-tokens', b'59931'), (b'x-ratelimit-reset-requests', b'16.359s'), (b'x-ratelimit-reset-tokens', b'69ms'), (b'x-request-id', b'req_229d5438c6499617740f4b430276155f'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'891c2d8aedbd01ce-CDG'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
2024-06-10 22:40:41 - INFO -

...

2024-06-10 22:40:42 - DEBUG - _trace.py:trace:45 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 10 Jun 2024 20:40:42 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-vycexojfpphtlaesrwlkvocv'), (b'openai-processing-ms', b'268'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'60000'), (b'x-ratelimit-remaining-requests', b'9997'), (b'x-ratelimit-remaining-tokens', b'59913'), (b'x-ratelimit-reset-requests', b'24.319s'), (b'x-ratelimit-reset-tokens', b'87ms'), (b'x-request-id', b'req_a3b5ff1fb4f3241803cda986d2299a7a'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'891c2d8f1d0201ce-CDG'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
2024-06-10 22:40:42 - INFO -

[llm/end] [chain:SequentialChain > chain:LLMChain > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "French",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "French",
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 1,
                "prompt_tokens": 78,
                "total_tokens": 79
              },
              "model_name": "gpt-3.5-turbo",
              "system_fingerprint": null,
              "finish_reason": "stop",
              "logprobs": null
            },
            "type": "ai",
            "id": "run-c656f7e4-aac9-41e2-abf1-d7349a139

2024-06-10 22:40:45 - DEBUG - _trace.py:trace:45 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 10 Jun 2024 20:40:45 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'user-vycexojfpphtlaesrwlkvocv'), (b'openai-processing-ms', b'2325'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'60000'), (b'x-ratelimit-remaining-requests', b'9996'), (b'x-ratelimit-remaining-tokens', b'59919'), (b'x-ratelimit-reset-requests', b'32.475s'), (b'x-ratelimit-reset-tokens', b'81ms'), (b'x-request-id', b'req_bc39d2a29e5544efbfa1550d08062be5'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'891c2d921a6a01ce-CDG'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
2024-06-10 22:40:45 - INFO 

[llm/end] [chain:SequentialChain > chain:LLMChain > llm:ChatOpenAI] [2.66s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Cher client,\n\nNous vous remercions pour vos commentaires concernant notre produit. Nous sommes désolés d'apprendre que vous avez été déçu par le goût moyen et le manque de tenue de la mousse. Nous tenons à vous assurer que notre produit est toujours fabriqué avec les meilleurs ingrédients de qualité. Cependant, nous allons immédiatement enquêter sur la question pour voir s'il s'agit d'un lot ancien ou contrefait. Votre satisfaction est importante pour nous et nous ferons tout notre possible pour remédier à cette situation.\n\nMerci encore pour votre retour et votre soutien.\n\nCordialement, [Votre société]",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": 

{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam does not hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed by the mediocre taste and lack of foam retention in the product, questioning whether it is an old batch or counterfeit.',
 'followup_message': "Cher client,\n\nNous vous remercions pour vos commentaires concernant notre produit. Nous sommes désolés d'apprendre que vous avez été déçu par le goût moyen et le manque de tenue de la mousse. Nous tenons à vous assurer que notre produit est toujours fabriqué avec les meilleurs ingrédients de qualité. Cependant, nous allons immédiatement enquêter sur la question pour voir s'il s'agit d'un lot ancien ou contrefait. Votre satisfaction est importante pour no

In [ ]:
import logging
import requests 


logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(filename)s:%(funcName)s:%(lineno)d - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
requests.packages.urllib3.add_stderr_logger()

In [223]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableMap

chain_one = first_prompt | llm | { "English_Review": StrOutputParser() }
#chain_two = second_prompt | llm | { "Review": itemgetter("Review"), "English_Review": RunnablePassthrough(), "summary": StrOutputParser() }
#chain_three = third_prompt | llm | {"Review": itemgetter("Review"), "English_Review": RunnablePassthrough(), "summary": RunnablePassthrough(), "language": StrOutputParser() }
#chain_four = fourth_prompt | llm | { "Review": itemgetter("Review"), "English_Review": RunnablePassthrough(), "summary": RunnablePassthrough(), "language": StrOutputParser(), "followup_message": StrOutputParser() }

chain_one = first_prompt | llm | StrOutputParser() # task: translate {Review} to {English_Review}
chain_two = {"English_Review": chain_one} | second_prompt | llm |  StrOutputParser() # task: summarize English_Review into {summary}
chain_three = {"summary": chain_two, "Review": itemgetter("Review")} | third_prompt | llm | StrOutputParser() # task: find {language} of {Review}
chain_four =  {"language": chain_three, "summary": chain_two} | fourth_prompt | llm | { "followup_message": StrOutputParser() } # task: generate follow up message to {summary} in {language}


In [225]:
import langchain
langchain.debug = False
review = df.Review[5]
chain_four.invoke({"Review": review})

2024-06-11 01:13:47 - DEBUG - _base_client.py:_build_request:446 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': "Translate the following review to English:\n\nJe trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?. Do not mention in the response that this is a translation.", 'role': 'user'}], 'model': 'llama3:8b', 'n': 1, 'stream': False, 'temperature': 0.0}}
2024-06-11 01:13:47 - DEBUG - _base_client.py:_request:949 - Sending HTTP Request: POST http://localhost:11434/v1/chat/completions
2024-06-11 01:13:47 - DEBUG - _trace.py:trace:45 - send_request_headers.started request=<Request [b'POST']>
2024-06-11 01:13:47 - DEBUG - _trace.py:trace:45 - send_request_headers.complete
2024-06-11 01:13:47 - DEBUG - _trace.py:trace:45 - send_request_body.started request=<Request [b'POST']>
2024-06-11 01:13:47 - DEBUG -

{'followup_message': "Bonjour,\n\nJe suis désolé d'apprendre que vous n'avez pas apprécié la saveur de votre bière achetée chez nous. Je comprends que le manque de tenue du mousse et la qualité médiocre de la bière peuvent être décevants.\n\nJe tiens à vous assurer que nous prenons très au sérieux vos commentaires et que nous allons enquêter sur les circonstances de votre achat. Il est possible que le lot de bière soit expiré ou qu'il y ait une erreur dans la production. Je vais également vérifier si votre produit est authentique.\n\nJe vous propose de me fournir plus d'informations sur votre achat, notamment la date et l'emplacement où vous l'avez acheté, ainsi que le code-barre du produit. Cela nous aidera à résoudre ce problème de manière efficace.\n\nJe suis à votre disposition pour discuter plus en détail de cette situation et trouver une solution qui vous convienne. N'hésitez pas à me contacter si vous avez des questions ou des préoccupations.\n\nCordialement,\n[Votre nom]\n\nTra

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
# class that wraps another class and logs all function calls being executed 
class Wrapper:
    def __init__(self, wrapped_class):
        self.wrapped_class = wrapped_class

    def __getattr__(self, attr):
        original_func = getattr(self.wrapped_class, attr)

        def wrapper(*args, **kwargs):
            print(f"Calling function: {attr}")
            print(f"Arguments: {args}, {kwargs}")
            result = original_func(*args, **kwargs)
            print(f"Response: {result}")
            return result

        return wrapper
    
llm.client = Wrapper(llm.client)


In [ ]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not \
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
import langchain.callbacks

langchain.callbacks.StdOutCallbackHandler

from FileCallbackHandler import FileCallbackHandler

from pathlib import Path

file_ballback_handler = FileCallbackHandler(Path('router_chain.txt'), print_prompts=True, )

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True, 
                         callbacks=[file_ballback_handler]
                        )

In [ ]:
chain.invoke("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")